In [49]:
pip install scikit-fuzzy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
import numpy as np
import pandas as pd

import skfuzzy as fuzz
from skfuzzy import control as ctrl

Generem dades artificials que de moment no tenim les oficials

In [51]:
# Llista dels 51 estats dels EUA (noms complets)
estats_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"]

# Genera dades aleatòries realistes (ajustables)
np.random.seed(42)  # Per reprodibilitat

# Genera valors aleatoris per hospitalitzats, morts i contagiats
df_epidemiologic = pd.DataFrame({
    "state": estats_usa,
    "hospitalitzats": np.random.randint(50, 2000, size=51),  # Exemple: entre 50 i 2000
    "morts": np.random.randint(5, 500, size=51),            # Exemple: entre 5 i 500
    "contagiats": np.random.randint(1000, 50000, size=51)   # Exemple: entre 1000 i 50000
})

# Mostra les primeres files
print(df_epidemiologic.head())

        state  hospitalitzats  morts  contagiats
0     Alabama            1176     55       34763
1      Alaska            1509    368       33606
2     Arizona             910     59       12534
3    Arkansas            1344    248       30127
4  California            1180    324       41397


In [52]:
df_demografic_original = pd.read_csv("../data/preprocessed/dataMatrix/static_stateMatrix.csv")
columnes = [
    "state", 
    "population_state",
    "pop_density_state",
    "bedsTotal",
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39",
    "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]
df_demografic = df_demografic_original[columnes]

Fusionem per tenir dades població (x edad ens poden servir també x tenir zones amb molta gent gran)

In [53]:
# Fusiona els dos DataFrames per la columna "state"
df_combinat = pd.merge(
    df_demografic, 
    df_epidemiologic, 
    on="state", 
    how="inner" 
)

# Mostra les primeres files (opcional)
print(df_combinat.head())

        state  population_state  pop_density_state  bedsTotal   pop_0-9  \
0     Alabama           4887871          96.509389        3.1  0.120933   
1      Alaska            737438           1.291523        2.2  0.142617   
2     Arizona           7171646          63.135855        1.9  0.123774   
3    Arkansas           3013825          57.919684        3.2  0.127289   
4  California          39557045         253.906502        1.8  0.124635   

   pop_10-19  pop_20-29  pop_30-39  pop_40-49  pop_50-59  pop_60-69  \
0   0.128108   0.136099   0.124291   0.123013   0.133249   0.120537   
1   0.130210   0.155501   0.147626   0.117347   0.127204   0.109072   
2   0.132496   0.140854   0.129224   0.119039   0.120016   0.113492   
3   0.132222   0.135421   0.126671   0.119787   0.126968   0.115755   
4   0.128760   0.147575   0.143412   0.128138   0.126974   0.104040   

   pop_70-79   pop_80+  hospitalitzats  morts  contagiats  
0   0.075243  0.038527            1176     55       34763  
1 

In [54]:
# IA (Incidència Acumulada)
df_combinat["IA"] = (df_combinat["contagiats"] / df_combinat["population_state"]) * 100000

# θ (Ocupació hospitalària %)
df_combinat["beds_real"] = (df_combinat["bedsTotal"] * df_combinat["population_state"]) / 1000
df_combinat["theta"] = (df_combinat["hospitalitzats"] / df_combinat["beds_real"]) * 100

# π (Taxa de mortalitat %) per 100000 habitants
df_combinat["pi"] = (df_combinat["morts"] / df_combinat["population_state"]) * 100000

# % població >65
df_combinat["pop_>65"] = df_combinat["pop_70-79"] + df_combinat["pop_80+"]
print(df_combinat.head())

        state  population_state  pop_density_state  bedsTotal   pop_0-9  \
0     Alabama           4887871          96.509389        3.1  0.120933   
1      Alaska            737438           1.291523        2.2  0.142617   
2     Arizona           7171646          63.135855        1.9  0.123774   
3    Arkansas           3013825          57.919684        3.2  0.127289   
4  California          39557045         253.906502        1.8  0.124635   

   pop_10-19  pop_20-29  pop_30-39  pop_40-49  pop_50-59  ...  pop_70-79  \
0   0.128108   0.136099   0.124291   0.123013   0.133249  ...   0.075243   
1   0.130210   0.155501   0.147626   0.117347   0.127204  ...   0.050675   
2   0.132496   0.140854   0.129224   0.119039   0.120016  ...   0.080306   
3   0.132222   0.135421   0.126671   0.119787   0.126968  ...   0.075878   
4   0.128760   0.147575   0.143412   0.128138   0.126974  ...   0.060900   

    pop_80+  hospitalitzats  morts  contagiats           IA   beds_real  \
0  0.038527      

In [55]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Definir variables d'entrada i sortida
ia = ctrl.Antecedent(np.arange(0, 10000, 1), 'ia')                  # Incidència acumulada (per 100k)
ocupacio = ctrl.Antecedent(np.arange(0, 101, 1), 'ocupacio')       # Ocupació hospitalària (%)
mortalitat = ctrl.Antecedent(np.arange(0, 101, 1), 'mortalitat')   # Taxa de mortalitat (per 100k)
poblacio65 = ctrl.Antecedent(np.arange(0, 101, 1), 'poblacio65')   # % població >65 anys
densitat = ctrl.Antecedent(np.arange(0, 15001, 1), 'densitat')     # Densitat (hab/km²)
risc = ctrl.Consequent(np.arange(0, 101, 1), 'risc')               # Risc de confinament (0-100)

# Funcions de membresia per a cada variable (ajustar rangs segons els teus datos)
# --- IA ---
ia['Baixa'] = fuzz.trimf(ia.universe, [0, 0, 50])       # Fins a 50
ia['Mitjana'] = fuzz.trimf(ia.universe, [30, 100, 150]) # Solapa amb Baixa i Alta
ia['Alta'] = fuzz.trimf(ia.universe, [130, 200, 10000])   # A partir de 150

# --- Ocupació Hospitalària ---
ocupacio['Baixa'] = fuzz.trimf(ocupacio.universe, [0, 0, 70])       # Fins a 70%
ocupacio['Moderada'] = fuzz.trimf(ocupacio.universe, [60, 75, 80])  # 70-80%
ocupacio['Crítica'] = fuzz.trimf(ocupacio.universe, [75, 90, 100])  # >80%

# --- Mortalitat ---
mortalitat['Baixa'] = fuzz.trimf(mortalitat.universe, [0, 0, 1])    # <1%
mortalitat['Mitjana'] = fuzz.trimf(mortalitat.universe, [0.5, 1.5, 2])  # 1-2%
mortalitat['Alta'] = fuzz.trimf(mortalitat.universe, [1.5, 3, 100])  # >2%

# --- Població >65 ---
poblacio65['Baixa'] = fuzz.trimf(poblacio65.universe, [0, 0, 10])    # <10%
poblacio65['Mitjana'] = fuzz.trimf(poblacio65.universe, [5, 15, 20]) # 10-20%
poblacio65['Alta'] = fuzz.trimf(poblacio65.universe, [18, 30, 30])   # >20%

# --- Densitat ---
densitat['Baixa'] = fuzz.trimf(densitat.universe, [0, 0, 1500])     # <75è percentil
densitat['Mitjana'] = fuzz.trimf(densitat.universe, [1000, 2000, 3000])  
densitat['Alta'] = fuzz.trimf(densitat.universe, [2500, 4000, 40000])  # >=75è percentil

# --- Risc ---
risc['Molt Baix'] = fuzz.trimf(risc.universe, [0, 0, 20])        # 0-20
risc['Baix'] = fuzz.trimf(risc.universe, [10, 25, 40])           # 20-40
risc['Moderat'] = fuzz.trimf(risc.universe, [30, 45, 60])        # 40-60
risc['Alt'] = fuzz.trimf(risc.universe, [50, 65, 80])            # 60-80
risc['Molt Alt'] = fuzz.trimf(risc.universe, [70, 85, 95])       # 80-95
risc['Extrem'] = fuzz.trimf(risc.universe, [90, 100, 100])       # 95-100

In [56]:
# --- Regles Principals ---
rule1 = ctrl.Rule(
    ia['Alta'] & ocupacio['Crítica'] & mortalitat['Alta'],
    risc['Extrem']
)
rule2 = ctrl.Rule(
    ia['Mitjana'] & ocupacio['Moderada'] & (poblacio65['Alta'] | densitat['Alta']),
    risc['Alt']
)
rule3 = ctrl.Rule(
    ia['Baixa'] & ocupacio['Baixa'] & mortalitat['Baixa'],
    risc['Molt Baix']
)
rule4 = ctrl.Rule(
    (ia['Mitjana'] | mortalitat['Mitjana']) & densitat['Mitjana'],
    risc['Moderat']
)
rule5 = ctrl.Rule(
    ocupacio['Crítica'] & (poblacio65['Alta'] | mortalitat['Alta']),
    risc['Molt Alt']
)

rule6 = ctrl.Rule(
    ia['Alta'] & ocupacio['Moderada'] & mortalitat['Mitjana'],
    risc['Alt']
)
rule7 = ctrl.Rule(
    ia['Mitjana'] & (poblacio65['Mitjana'] | densitat['Mitjana']),
    risc['Moderat']
)
rule8 = ctrl.Rule(
    mortalitat['Alta'] & (ia['Baixa'] | ocupacio['Baixa']),
    risc['Alt']
)
rule9 = ctrl.Rule(
    densitat['Alta'] & ia['Mitjana'],
    risc['Moderat']
)

# --- Regla Per Defecte (Captura Casos Residuals) ---
rule10 = ctrl.Rule(
    ia['Baixa'] | ocupacio['Baixa'] | mortalitat['Baixa'],
    risc['Baix']
)

# Crear sistema de control
risc_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, 
    rule6, rule7, rule8, rule9, rule10])



In [57]:
df_combinat.columns


Index(['state', 'population_state', 'pop_density_state', 'bedsTotal',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+', 'hospitalitzats',
       'morts', 'contagiats', 'IA', 'beds_real', 'theta', 'pi', 'pop_>65'],
      dtype='object')

In [58]:
print("Màxim IA:", df_combinat['IA'].max())
print("Màxim Theta:", df_combinat['theta'].max())
print("Màxim Densitat:", df_combinat['pop_density_state'].max())

Màxim IA: 7534.570228321883
Màxim Theta: 93.01244184719134
Màxim Densitat: 11490.11954


In [59]:
def calcular_risc(row):
    risc_sim = ctrl.ControlSystemSimulation(risc_ctrl)
    inputs = {
        'ia': row['IA'],
        'ocupacio': row['theta'],
        'mortalitat': row['pi'],
        'poblacio65': row['pop_>65'],
        'densitat': row['pop_density_state']
        }
    for key, val in inputs.items():
        risc_sim.input[key] = val
    risc_sim.compute()
    return risc_sim.output['risc']

# Aplicar UNA VEGADA
df_combinat['nivell_risc'] = df_combinat.apply(calcular_risc, axis=1)

# Classificar decisió segons els nous llindars
df_combinat['confinament'] = np.select(
    [
        df_combinat['nivell_risc'] < 40,  # Si nivell_risc < 40
        (df_combinat['nivell_risc'] >= 40) & (df_combinat['nivell_risc'] < 60),  # Si nivell_risc entre 40 i 60
        (df_combinat['nivell_risc'] >= 60) & (df_combinat['nivell_risc'] < 80),  # Si nivell_risc entre 60 i 80
        df_combinat['nivell_risc'] >= 80  # Si nivell_risc >= 80
    ],
    [
        'No',        # nivell_risc < 40
        'Selectiu',     # nivell_risc entre 40 i 60
        'Estricte',  # nivell_risc entre 60 i 80
        'Imediat'   # nivell_risc >= 80
    ],
    default='No'  # Valor per defecte (opcional)
)

In [ ]:
df_combinat['bedsTotal2'] = (df_combinat['bedsTotal']/1000) * df_combinat['population_state']
df_combinat['bedsTotal3'] = df_combinat['bedsTotal2'] - df_combinat['hospitalitzats']

NameError: name 'df_combinat' is not defined

In [ ]:
df_veins = pd.read_csv("../data/raw/neighbor_states/neighbor_states.csv")

In [ ]:
df_combinat